튜토리얼을 위해 노트북으로 작성하였습니다. 유사한 시험환경을 위해 스크립트 모드로 작성해 보세요!!

In [ ]:
################## 시험 안내 문구 및 코드 ##################
# 출력을 원하실 경우 print() 함수 활용
# 예시) print(df.head())

# getcwd(), chdir() 등 작업 폴더 설정 불필요
# 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

# 데이터 파일 읽기 예제
import pandas as pd
X_test = pd.read_csv("../input/hr-data/X_test.csv")
X_train = pd.read_csv("../input/hr-data/X_train.csv")
y_train = pd.read_csv("../input/hr-data/y_train.csv")

# 사용자 코딩

# 답안 제출 참고
# 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
# pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': pred}).to_csv('003000000.csv', index=False)

# EDA

In [ ]:
# 데이터 크기 확인
X_train.shape, y_train.shape, X_test.shape

In [ ]:
# 데이터 샘플 확인
X_train.head()

In [ ]:
# 예측 해야할 타겟 샘플 확인 (0과 1이지만 문제에서는 확률값으로 예측하라고 되어 있음)
y_train.head()

In [ ]:
# 예측할 데이터 균형 확인
y_train['target'].value_counts()

In [ ]:
# 데이터 타입 확인
print(X_train.info())

In [ ]:
# 결측치 확인
print("##Train##")
print(X_train.isnull().sum())
print("\n##Test##")
print(X_test.isnull().sum())

In [ ]:
# 이상치IQR
# training_hours
import numpy as np
Q1 = np.percentile(X_train['training_hours'],25)
Q3 = np.percentile(X_train['training_hours'],75)
IQR = Q3 - Q1
outdata1 = X_train[X_train['training_hours']<(Q1 - 1.5 * IQR)]
outdata2 = X_train[X_train['training_hours']>(Q3 + 1.5 * IQR)]
len(outdata1), len(outdata2)
# 삭제하긴 많음

In [ ]:
# city_development_index
Q1 = np.percentile(X_train['city_development_index'],25)
Q3 = np.percentile(X_train['city_development_index'],75)
IQR = Q3 - Q1
outdata1 = X_train[X_train['city_development_index']<(Q1 - 1.5 * IQR)]
outdata2 = X_train[X_train['city_development_index']>(Q3 + 1.5 * IQR)]
len(outdata1), len(outdata2)
#삭제 예정

In [ ]:
# 데이터 확인
obj_cols = np.array(X_train.columns[X_train.dtypes == object])
for col in obj_cols:
    print("\n##### : ",col)
    print(X_train[col].value_counts())
obj_cols

# 데이터 전처리

In [ ]:
# 심플 결측치 처리
X_train = X_train.fillna("X")
X_test = X_test.fillna("X")

In [ ]:
# 이상치 삭제
print(X_train.shape)
ind = X_train[X_train['city_development_index']<(Q1 - 1.5 * IQR)].index
X_train = X_train.drop(index=ind, axis=0)
y_train = y_train.drop(index=ind, axis=0)

print(X_train.shape)

# 피처 엔지니어링

In [ ]:
from sklearn.preprocessing import LabelEncoder

all_df = pd.concat([X_train.assign(ind="train"), X_test.assign(ind="test")])
le = LabelEncoder()
all_df[obj_cols] = all_df[obj_cols].apply(le.fit_transform)

X_train = all_df[all_df['ind'] == 'train']
X_train = X_train.drop('ind',axis=1)
X_train

X_test = all_df[all_df['ind'] == 'test']
X_test = X_test.drop('ind',axis=1)
X_test

In [ ]:
# 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()

n_cols = ['city_development_index', 'training_hours']
X_train[n_cols] = scaler.fit_transform(X_train[n_cols])
X_test[n_cols] = scaler.transform(X_test[n_cols])
X_train

# 데이터 분리

In [ ]:
# 학습용 데이터와 검증용 데이터로 구분
# 학습에 많은 데이터를 사용하기 위해 검증은 작게 가져감
from sklearn.model_selection import train_test_split


X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['target'], test_size=0.1, random_state=2022)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

# 모델 학습 및 평가

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
rf = RandomForestClassifier(random_state = 2022)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)[:,1]
print(roc_auc_score(y_val, pred))

# 제출용

In [ ]:
# Test데이터
pred = rf.predict_proba(X_test)[:,1]

In [ ]:
# csv생성코드
pd.DataFrame({'enrollee_id': X_test.enrollee_id, 'target': pred}).to_csv('003000000.csv', index=False)

# 체점

In [ ]:
# 체점
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score

with open( "../input/hr-data/answer.pickle", "rb" ) as file:
    ans = pickle.load(file)
    ans = pd.DataFrame(ans)
print(roc_auc_score(ans['target'], pred))